# Collect data from Spotify and Last.fm

In [1]:
from crawler import Crawler

crawler = Crawler("127.0.0.1", 5000)

In [2]:
crawler.collect_tracks(1000)

No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
Error while fetching audio analysis.
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
Error while fetching audio analysis.
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
Error while fetching audio analysis.
Error while fetching audio analysis.
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
Error while fetching audio analysis.
No tags 116t6KTSHuNc4ZRm8arZQ7 o% ['Niil B']
No tags 116

KeyboardInterrupt: 